In [41]:
import pandas as pd



In [42]:
df = pd.read_csv("Ton_IoT_Network.csv")
print(df.shape)
print(df.head(10))


(461043, 45)
           ts         src_ip  src_port         dst_ip  dst_port proto service  \
0  1554198358    3.122.49.24      1883  192.168.1.152     52976   tcp       -   
1  1554198358   192.168.1.79     47260  192.168.1.255     15600   udp       -   
2  1554198359  192.168.1.152      1880  192.168.1.152     51782   tcp       -   
3  1554198359  192.168.1.152     34296  192.168.1.152     10502   tcp       -   
4  1554198362  192.168.1.152     46608  192.168.1.190        53   udp     dns   
5  1554198364   192.168.1.79     33269  192.168.1.255     15600   udp       -   
6  1554198364  192.168.1.152     34296  192.168.1.152     10502   tcp       -   
7  1554198364  192.168.1.152      1880  192.168.1.152     51782   tcp       -   
8  1554198369  192.168.1.152      1880  192.168.1.152     51782   tcp       -   
9  1554198369  192.168.1.152     34296  192.168.1.152     10502   tcp       -   

       duration  src_bytes  dst_bytes conn_state  missed_bytes  src_pkts  \
0  80549.530260    

In [43]:


# Loại bỏ dòng trùng
df = df.drop_duplicates()

# Xóa giá trị thiếu hoàn toàn
df = df.dropna(how='all')

# Thay thế dấu "-" (ý là không có dữ liệu)
df = df.replace('-', pd.NA)


In [44]:
print(df.shape)

(449972, 45)


In [45]:
cols_to_drop = [
    'src_ip', 'dst_ip', 'ssl_subject', 'ssl_issuer', 
    'weird_addl', 'weird_name', 'weird_notice', 
    'dns_query'
]
df = df.drop(columns=cols_to_drop, errors='ignore')


In [46]:
print(df.shape)

(449972, 37)


In [47]:
df['ts'] = pd.to_datetime(df['ts'], unit='s')  # chuyển timestamp thành datetime
df['hour'] = df['ts'].dt.hour
df['day'] = df['ts'].dt.day
df = df.drop(columns=['ts'])


In [48]:
cat_cols = df.select_dtypes(include='object').columns
print(cat_cols)


Index(['proto', 'service', 'conn_state', 'dns_AA', 'dns_RD', 'dns_RA',
       'dns_rejected', 'ssl_version', 'ssl_cipher', 'ssl_resumed',
       'ssl_established', 'http_trans_depth', 'http_method', 'http_uri',
       'http_version', 'http_user_agent', 'http_orig_mime_types',
       'http_resp_mime_types', 'type'],
      dtype='object')


In [49]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col].astype(str))


In [50]:
from sklearn.preprocessing import StandardScaler

num_cols = df.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])


In [51]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['label', 'type'])  # input features
y = df['label']                         # target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [52]:
y.value_counts(normalize=True)


label
-0.746578    0.642104
 1.339444    0.357896
Name: proportion, dtype: float64

In [ ]:
processed = pd.concat([X, y], axis=1)
processed.to_csv("Ton_IoT_Network_processed.csv", index=False)
